In [4]:
def parse_custom_data(file_path):
    """Parse the custom formatted configuration file."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    cell = []
    atoms = []
    reading_supercell = False
    reading_atoms = False

    for line in lines:
        stripped_line = line.strip()
        if 'Supercell' in stripped_line:
            reading_supercell = True
            continue  # Ensure we do not process 'Supercell' as data
        elif 'AtomData:' in stripped_line:
            reading_supercell = False  # Ensure to stop reading Supercell if still on
            reading_atoms = True
            continue  # Skip the 'AtomData:' header line
        elif stripped_line.startswith('END_CFG') or not stripped_line:
            reading_supercell = False
            reading_atoms = False
        elif reading_supercell:
            try:
                cell_dims = [float(x) for x in stripped_line.split()]
                cell.append(cell_dims[cell_dims.index(max(cell_dims))])  # Add only the nonzero dimension
            except ValueError:
                # Encountered a line that's not numeric, which indicates the end of the Supercell section
                reading_supercell = False
        elif reading_atoms and stripped_line:
            parts = stripped_line.split()
            if len(parts) == 9:  # Ensure this is a valid atom data line
                atom_data = {
                    'id': int(parts[0]),
                    'type': int(parts[1]),
                    'x': float(parts[2]),
                    'y': float(parts[3]),
                    'z': float(parts[4]),
                    'fx': float(parts[5]),
                    'fy': float(parts[6]),
                    'fz': float(parts[7]),
                    'nbh_grades': float(parts[8])
                }
                atoms.append(atom_data)

    return cell, atoms

def write_lammps_dump(output_path, cell, atoms):
    """Write data to a LAMMPS dump file format."""
    with open(output_path, 'w') as file:
        file.write("ITEM: TIMESTEP\n0\n")
        file.write("ITEM: NUMBER OF ATOMS\n")
        file.write(f"{len(atoms)}\n")
        file.write("ITEM: BOX BOUNDS pp pp pp\n")
        for dim in cell:  # Print each dimension's bounds from zero to the dimension value
            file.write(f"0.0 {dim}\n")
        file.write("ITEM: ATOMS id type x y z fx fy fz nbh_grades\n")
        for atom in atoms:
            file.write(f"{atom['id']} {atom['type']} {atom['x']} {atom['y']} {atom['z']} ")
            file.write(f"{atom['fx']} {atom['fy']} {atom['fz']} {atom['nbh_grades']}\n")

# Example usage
for i in range(5):
    input = str(i+1)+'-grade.cfg.0'
    cell, atoms = parse_custom_data(input)
    output = str(i+1)+'-cfg.dump'
    write_lammps_dump(output, cell, atoms)
